In [40]:
#On va importer les packages 
!pip install wordcloud 
!pip install nltk
!pip install pattern

import pandas as pd
import re
import os

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

import matplotlib.pyplot as plt
from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

  Using cached Pattern-3.6.0.tar.gz (22.2 MB)
  Preparing metadata (setup.py) ... done
  Using cached future-0.18.3.tar.gz (840 kB)
  Preparing metadata (setup.py) ... done
  Using cached backports.csv-1.0.7-py2.py3-none-any.whl (12 kB)
  Using cached mysqlclient-2.2.1.tar.gz (89 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [27 lines of output]
      /bin/sh: 1: pkg-config: not found
      /bin/sh: 1: pkg-config: not found
      /bin/sh: 1: pkg-config: not found
      Trying pkg-config --exists mysqlclient
      Command 'pkg-config --exists mysqlclient' returned non-zero exit status 127.
      Trying pkg-config --exists mariadb
      Command 'pkg-config --exists mariadb' returned non-zero exit status 127.
      Trying pkg-config --exists libmariadb
      Command 'pkg-config --exists libmariadb' returned non-zer

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [93]:
#Récupérations données 
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers le répertoire de données
fichier = os.path.join(base_dir, 'data', 'base_def.csv')

df = pd.read_csv(fichier)

In [ ]:
# Travail des stop words
mots_fr = [
    'le', 'la', 'les', 'de', 'un', 'à', 'être', 'suis', 'es', 'est', 'sont', 'été', 'étais', 'était', 'étaient', 'serai', 'seras', 'sera', 'seront',
    'et', 'en', 'avoir', 'ai', 'as', 'a', 'ont', 'eu', 'aurai', 'auras', 'aura', 'auront', 'avais', 'avait', 'avaient', 'que', 'pour',
    'dans', 'ce', 'il', 'qui', 'ne', 'en', 'sur', 'se', 'pas', 'plus',
    'par', 'je', 'avec', 'tout', 'faire', 'fais', 'fait', 'font', 'son', 'mettre', 'autre', 'on', 'mais',
    'nous', 'comme', 'ou', 'si', 'leur', 'y', 'dire', 'dis', 'dit', 'disent' 'elle', 'voir', 'devoir',
    'deux', 'même', 'pendant', 'aussi', 'vouloir', 'grand', 'mon', 'nouveau', 'aller',
    'venir', 'ceux', 'faire', 'quelque', 'trouver', 'donner', 'donne', 'donnes', 'donnent', 'aussi', 'autre',
    'tous', 'vieux', 'bon', 'voir', 'moins', 'trois', 'avant', 'sa', 'faire', 'contre',
    'abord', 'sous', 'ou', 'apporter', 'grand', 'ainsi', 'long', 'très', 'tout', 'avoir',
    'beau', 'chaque', 'peu', 'quoi', 'encore', 'aller', 'montrer', 'semaine', 'ainsi',
    'nuit', 'aussi', 'bien', 'deuxième', 'moins', 'tout', 'avoir', 'peu', 'nuit', 'ça',
    'mon', 'ma', 'mes', 'ton', 'ta', 'tes', 'son', 'sa', 'ses', 'notre', 'notre', 'nos', 'votre', 'votre', 'vos', 'leur', 'leur', 'leurs',
    'non', '’', '.', ',', ':', 'est', 'des', "l'", "d'", "une", "d", "»", "«", "du", "qu", "au", "n", "s", "elle", ")", "(", "aux", 'c', 'cette', 'un', 'ces','?', 'il', 'ils', 'j', "l", "-", "vous", "entre",
    "[", "]", "elles", '”', '“', "dont", "cela", ";", "m", "selon", "!", '–'
]
# Concaténer tous les textes de la colonne 'contenu' en une seule chaîne
all_text = ' '.join(df['contenu'])

# Fonction pour enlever les mots de la liste mots_fr
def remove_words(text):
    words = word_tokenize(text.lower())  # normaliser la casse ici
    filtered_words = [word for word in words if word not in mots_fr]
    return ' '.join(filtered_words)
    
# Appliquer la fonction remove_words
df['contenu'] = df['contenu'].apply(lambda x: remove_words(x))

# Fonction pour normaliser les mots au singulier [va falloir singulariser autrement]
def singularize_word(word):
    # Retire le 's' à la fin du mot si présent et si la version sans 's' existe déjà strictement
    if word.lower().endswith('s') and word[:-1].lower() in set(all_text.split()):
        return word[:-1]
    else:
        return word
        
df['contenu'] = df['contenu'].apply(lambda x: ' '.join([singularize_word(word) for word in x.split()]))

all_text2 = ' '.join(df['contenu'])

# Fonction pour obtenir les mots les plus représentés
def get_most_common_words(text):
    words = word_tokenize(text.lower())
    freq_dist = FreqDist(words)
    return freq_dist.most_common()

# Appliquer la fonction à l'ensemble du texte
most_common_words = get_most_common_words(all_text2)
most_common_words